[![pythonista](img/pythonista.png)](https://www.pythonista.io)

# Las claúsulas `PIVOT` y `UNPIVOT`.

In [2]:
from pyspark.sql import SparkSession
import pandas as pd
import numpy as np
spark = SparkSession.builder.appName("pivot y unpivot").getOrCreate()
ct = spark.sparkContext
%load_ext sparksql_magic

## La claúsula `PIVOT`.

La claúsula `PIVOT` permite generar nuevas estructuras tabulares, mediante la creación de nuevas columnas mediante agrupación y funciones de agregación.

```
SELECT 
    <col_pref_suf_1>,
    <col_pref_suf_2>,
    ...
    ... 
    <col_pref_suf_n>
FROM <fuente>
PIVOT(
    <func_1>(<col_agr_1>) AS <sufijo_1>,
    <func_2>(<col_agr_2>) AS <sufijo_2>,
    ...
    ...
    <func_n>(<col_agr_n>) AS <sufijo_n>
    FOR <col_grp>
    IN (
         <dato_pivote_1> AS prefijo_1>,
         <dato_pivote_2> AS prefijo_2>,
         ...
         ...
         <dato_pivote_3> AS prefijo_3>
         )
    )
```

Donde:

* `<fuente>` Es la fuente datos.
* `<func_x>` Es una función de agregación.
* `<col_agr_x>` Es una columna a la que se le aplicará una función de agregación.
* `<sufijo_x>` Es una cadena de caracteres que se utilizará como sufijo del nombre de las nuevas columnas.
* `<col_grp>` Es la columna a partir del cual se realizará un agrupamiento similar a `GROUP BY`.
* `<dato_pivote_x>` Es un dato dentro de `<col_grp>` que será agrupado.
* `<prefijo_x>` Es una cadena de caracteres que se utilizará como prefijo del nombre de las nuevas columnas.

    Para poder realizar la agrupación, es necesario que el  contenido de los campos distintos a las columnas utilizadas por `PIVOT` sean iguales.

La documentación esta disponible en:

* https://spark.apache.org/docs/latest/sql-ref-syntax-qry-select-pivot.html

## Ejemplos ilustrativos de `PIVOT`.

### Caso de un censo simple.

* Se creará la tabla `censo simple` en la que existen las columnas:
    * `region`, la cual contiene los valores agrupables:
        * `Norte`
        * `Sur`
        * `Este`
        * `Oeste`
    * `conejos`, la cual contiene números enteros.

In [3]:
(spark.createDataFrame(pd.DataFrame({
    'region':(
        'Norte',
        'Sur',
        'Este',
        'Oeste',
        'Norte',
        'Este',
        'Norte',
        'Sur'),
    'conejos':(
        153, 
        289,
        35,
        215,
        91,
        52,
        128,
        94
    )
}))
 .createOrReplaceTempView('censo_simple'))

/opt/conda/lib/python3.11/site-packages/pyspark/sql/pandas/conversion.py:479: FutureWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if should_localize and is_datetime64tz_dtype(s.dtype) and s.dt.tz is not None:


In [4]:
%%sparksql
SELECT * 
FROM censo_simple;

region,conejos
Norte,153
Sur,289
Este,35
Oeste,215
Norte,91
Este,52
Norte,128
Sur,94


* La siguiente consulta creará una tabla con las columnas:
    * Norte
    * Sur
    * Este
    * Oeste
* Cada columna contendrá la suma (al apicarse la función `sum()`) de los valores de la columna `conejos` agrupados.

In [5]:
%%sparksql --view poblacion_simple
SELECT Norte, Sur, Este, Oeste
FROM censo_simple
PIVOT(
    SUM(conejos) 
    FOR region 
    IN (
        'Norte', 
        'Sur', 
        'Este', 
        'Oeste'
    )
);

create temporary view `poblacion_simple`


Norte,Sur,Este,Oeste
372,383,87,215


### Caso de un censo de múltiples especies.

* Se creará la tabla `censo` en la que existen las columnas:
    * `region`, la cual contiene los valores agrupables:
        * `Norte`
        * `Sur`
        * `Este`
        * `Oeste`
    * `conejos`, la cual contiene números enteros.
    * `liebres`, la cual contiene números enteros.

In [6]:
(spark.createDataFrame(pd.DataFrame({
    'region':(
        'Norte',
        'Sur',
        'Este',
        'Oeste',
        'Norte',
        'Este',
        'Norte',
        'Sur'),
    'conejos':(
        153, 
        289,
        35,
        215,
        91,
        52,
        128,
        94),
    'liebres':(
        25,
        56,
        63,
        12,
        25,
        29,
        23,
        32)
})).createOrReplaceTempView('censo'))

In [7]:
%%sparksql
SELECT * 
FROM censo;

region,conejos,liebres
Norte,153,25
Sur,289,56
Este,35,63
Oeste,215,12
Norte,91,25
Este,52,29
Norte,128,23
Sur,94,32


* La siguiente celda ejecutará una consulta con `PIVOT` aplicando la función de agregación `SUM()` sobre `conejos` únicamente.
* `PIVOT` sólo puede agrupar registros idénticos, pero ninguno de los datos en la columna `liebres` es igual.

In [8]:
%%sparksql
SELECT Norte, Sur, Este, Oeste
FROM censo
PIVOT(
    SUM(conejos) 
    FOR region 
    IN (
        'Norte', 
        'Sur', 
        'Este', 
        'Oeste'
    )
);

Norte,Sur,Este,Oeste
null,null,52,null
null,94,null,null
244,null,null,null
null,null,35,null
null,289,null,null
null,null,null,215
128,null,null,null


* La siguiente celda ejecutará una consulta con `PIVOT` aplicando la función de agregación `SUM()` sobre `conejos` y la función de agregación `SUM()` sobre `liebres`.
* Se utilizan alias para renombrar las tablas.

In [9]:
%%sparksql --view poblacion
SELECT 
    N_c,
    S_c, 
    E_c,
    W_c,
    N_l,
    S_l,
    E_l,
    W_l
FROM censo
PIVOT (
    SUM(conejos) as c,
    SUM(liebres) as l
    FOR region 
    IN (
        'Norte' AS N, 
        'Sur' AS S, 
        'Este' AS E, 
        'Oeste' AS W
    )
);

create temporary view `poblacion`


N_c,S_c,E_c,W_c,N_l,S_l,E_l,W_l
372,383,87,215,73,88,92,12


## La claúsula `UNPIVOT`.

```
SELECT <columnas>
FROM <fuente>
UNPIVOT (
    <columna_data>
    FOR <columna_condensada>
    IN (
        <columna_parcial_1> AS <valor_1>,
        <columna_parcial_2> AS <valor_2>,
        ...
        ...
        <columna_parcial_n> AS <valor_n>
    )
);
```

Donde:
* `<fuente>` es la tabla de rogien de los datos. 
* `<columna_parcial_i>` es una de varias columna que contiene los datos a consolidar.
* `<columna_data>` es el nombre de una nueva columna en la que se consolidarán los datos.
* `<columna_condensada>` es el nombre de una nueva columna en la que se distribuirán los nombres de las columnas parciales.


https://spark.apache.org/docs/latest/sql-ref-syntax-qry-select-unpivot.html

## Ejemplos ilustrativos de `UNPIVOT`.

### Uso de la tabla `poblacion_simple`.

In [10]:
%%sparksql
SELECT *
FROM poblacion_simple;

Norte,Sur,Este,Oeste
372,383,87,215


* La siguiente celda regresará una tabla con las columnas:
    * `zona`, con cada zona correspondiente.
    * `conejos`, con el consolidado de datos.

In [11]:
%%sparksql
SELECT *
FROM poblacion_simple
UNPIVOT (
    conejos 
    FOR zona 
    IN (
        Norte,
        Sur,
        Este,
        Oeste
    )
);

zona,conejos
Norte,372
Sur,383
Este,87
Oeste,215


### Uso de la tabla `poblacion`.

In [13]:
%%sparksql
SELECT *
FROM poblacion;

N_c,S_c,E_c,W_c,N_l,S_l,E_l,W_l
372,383,87,215,73,88,92,12


In [ ]:
%%sparksql
SELECT *
FROM poblacion
UNPIVOT (
    conejos
    FOR zona 
    IN (
        N_c, 
        S_c,
        E_c,
        W_c
    )
);

In [ ]:
%%sparksql
SELECT zona, conejos
FROM poblacion
UNPIVOT (
    conejos
    FOR zona 
    IN (
        N_c as Norte, 
        S_c as Sur,
        E_c as Este,
        W_c as Oeste
    )
);

In [ ]:
spark.stop()

<p style="text-align: center"><a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Licencia Creative Commons" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/80x15.png" /></a><br />Esta obra está bajo una <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Licencia Creative Commons Atribución 4.0 Internacional</a>.</p>
<p style="text-align: center">&copy; José Luis Chiquete Valdivieso. 2023.</p>